##### This is where I will be brining in Coinmarketcap.com data for ranks, coins, stats, new coins, and trends

### Set up

In [130]:
from pymarketcap import *  
import json
import pprint
import pandas as pd
from datetime import datetime, timedelta 
from bs4 import BeautifulSoup
import urllib
import time
import logging
from apscheduler.schedulers.blocking import BlockingScheduler
pd.options.display.float_format = '{:20,.2f}'.format
#Used for saving files
moment=time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())

#Connects to coinmarketcap API
coinmarketcap = Pymarketcap()

#Used for todays date
today = pd.Timestamp("today").strftime("%m/%d/%Y")
hh = pd.Timestamp("today").strftime("%H")
mm = pd.Timestamp("today").strftime("%M")


### Define function that allows replacement for new coin date added

In [131]:
def setdate(x):
    if len(x) == 5 or len(x) == 7:
        return datetime.today().strftime("%m/%d/%Y")
    elif len(x) == 8:
        newDate = datetime.today() - timedelta(days=int(x[:1]))
        return newDate.strftime("%m/%d/%Y")
    else:
        newDate = datetime.today() - timedelta(days=int(x[:2]))
        return newDate.strftime("%m/%d/%Y")

### Coinmarketcap statistics

In [132]:
try:
    #Bring data into stats
    stats = json.loads(coinmarketcap.stats(VERBOSE=True))

    #bring data into csv
    stats = pd.DataFrame(statistics)

    #add 'Today' column
    stats['Date'] = today
    stats['Hour'] = hh
    stats['Minute'] = mm

    #Save CSV
    stats.to_csv('Coinmarketcap/stats_'+moment+'.csv', sep=',') 
except:
    pass

### Coin Data

In [133]:
try:
    #Bring data into coinsdata
    ticker = json.loads(coinmarketcap.ticker(VERBOSE=True))

    #bring data into csv
    ticker = pd.DataFrame(ticker)

    #add 'Today' 'Hour' and 'minutes' column
    ticker['Date'] = today
    ticker['Hour'] = hh
    ticker['Minute'] = mm

    ticker
    #Save CSV
    ticker.to_csv('Coinmarketcap/coins_'+moment+'.csv', sep=',')  
except:
    pass


### Each coin's exchanges info

In [134]:
for coin2 in ticker['symbol']:
    try: 
        #Bring data into coinsdata
        markets = json.loads(coinmarketcap.markets(coin2,VERBOSE=True))

        #bring data into csv
        markets = pd.DataFrame(markets)

        #add 'Today' 'Hour' and 'minutes' column
        markets['Date'] = today
        markets['Hour'] = hh
        markets['Minute'] = mm


        #Save CSV
        markets.to_csv('Coinmarketcap/markets_'+coin2+"_"+moment+'.csv', 'w') 
    except:
        pass

### Ranks for coins

In [135]:
try: 
    #You can pass '7d', '24h', '1h', 'gainers' and 'losers' for filter responses:
    #coinmarketcap.ranks('gainers', '1h', V=True)
    #Bring data into coinsdata
    ranks = json.loads(coinmarketcap.ranks(V=True))

    #bring data into csv
    ranks = pd.DataFrame(ranks)

    #add 'Today' 'Hour' and 'minutes' column
    ranks['Date'] = today
    ranks['Hour'] = hh
    ranks['Minute'] = mm


    #Save CSV
    markets.to_csv('Coinmarketcap/ranks_'+moment+'.csv', 'w')
except:
    pass

In [ ]:
# ### New coins

# In[ ]:

#Grabs new coins from coinmarket cap and saves as CSV

def new_coin(): 
    moment=time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())
    r = urllib.urlopen('http://coinmarketcap.com/new/').read()
    soup = BeautifulSoup(r, "lxml")
    col = soup.find('tr').find_all('td')
    for c in col:
        print(c.get_text())
    final_list = []
    tr = soup.findChildren('tr') 
    final_list = []
    for t in tr:  
        td = t.findChildren('td')  
        coin_list = []
        for child in td:
            coin_list.append(child.text.replace('\n', '').strip().replace(" ", ""))
        if coin_list != []:
            final_list.append(coin_list)

    data = pd.DataFrame(final_list, columns = ["Name","Symbol", "Added", "Market Cap","Price","Circulating Supply","Volume (24h)", "% 24h"])        
    data["Added"] = data["Added"].apply(lambda x: setdate(x))
    data['Date'] = today
    data['Hour'] = hh
    data['Minute'] = mm
    data.to_csv('Coinmarketcap/new_'+moment+'.csv', sep=',')  

new_coin()









In [10]:
# ### Run block scheduler    

# In[ ]:

scheduler = BlockingScheduler()
scheduler.add_job(new_coin, 'interval', minutes=60, id="coin")
scheduler.add_job(grab_info, 'interval', minutes=60, id="info")
scheduler.start()
#while True:
 #   time.sleep(60)
#sched.shutdown()